In [4]:
import annotation_utils as utils
import numpy as np
import cv2
np.set_printoptions(suppress=True)
import warnings
warnings.filterwarnings('ignore')

camera_params = [633.0902963939839, 638.754439101168,
                 629.0646296262861, 362.7401326185789]

keypoints, anchors = utils.get_points_from_CAD()

normalized_keypoints = utils.normalize_and_orient_kpts(keypoints)
normalized_anchors = utils.normalize_and_orient_kpts(anchors)

anchor_ground_truth = np.array(
([[606, 323],
       [571, 277],
       [642, 277],
       [536, 323],
       [676, 322],
       [571, 383],
       [640, 383]]))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 2)

In [29]:
from skimage.transform import ProjectiveTransform, warp
from skimage import transform
from scipy.spatial.transform import Rotation as R

def get_corrected_normalized_keypoints(res_warp,
                                        anchors_points_normalized,
                                        anchor_ground_truth,
                                        normalized_keypoints):
    

    def get_corrective_tf(anchor_marker, anchor_gt):
        corrective_tf = cv2.estimateAffine2D(anchor_marker, anchor_gt,ransacReprojThreshold = 2)
        return corrective_tf
    
    def debug_pts(keypoints, comment):
        no_marker_name = 'no_marker_close_new_2.jpg'
        img_nm = cv2.imread(no_marker_name)

        for kp in keypoints:
            cv2.circle(img_nm, (int(kp[0]), int(kp[1])), 2, (0,0,255), -1)
        cv2.imshow(comment, img_nm)
        cv2.waitKey(0) 
    
    anchors_points = get_keypoints_on_warped_image(res_warp, anchors_points_normalized)
    keypoints = get_keypoints_on_warped_image(res_warp, normalized_keypoints)
    # print(keypoints)
    corrective_tf, inliers = get_corrective_tf(anchors_points,anchor_ground_truth)
    
#     tform = ProjectiveTransform()
    tform = transform.estimate_transform('euclidean', anchors_points,anchor_ground_truth)
#     tform.estimate('euclidean',anchors_points,anchor_ground_truth)
    print("tform.params")
    print(tform.params)
    print("resulting transform to correct points")
    print(corrective_tf)
    print("inliers", inliers)
    dx = (res_warp[0].corners[2] - res_warp[0].corners[1])
    center = (res_warp[0].corners[1] + res_warp[0].corners[3])/2
    scale = dx[0]
    homog_keypoints = np.hstack((keypoints,np.ones((keypoints.shape[0],1))))
    # print(tform.params @ homog_keypoints.T)
    corrective_tf = tform.params[:2]
    corrected_points =(corrective_tf @ homog_keypoints.T).T
    # print(corrected_points)
    # print((corrective_tf @ homog_keypoints.T).T)
    corrective_tf = tform.params[:2]
    transformed_anchors_points = (corrective_tf @  np.hstack((anchors_points,np.ones((anchors_points.shape[0],1)))).T).T

    debug_pts(anchor_ground_truth,"anchor_ground_truth")

#     for p in corrected_points:
# #         print(p)
#         p/=p[2]

    cv2.destroyAllWindows()
    
    corrected_normalized_kpts =  (corrected_points - center)/ scale

    return corrected_points, corrected_normalized_kpts,transformed_anchors_points


def get_keypoints_on_warped_image(res_warp, normalized_kpts):
    dx = (res_warp[0].corners[2] - res_warp[0].corners[1])
    center = (res_warp[0].corners[1] + res_warp[0].corners[3])/2
    scale = dx[0] 
    keypoints = normalized_kpts * scale + center
    
    return keypoints


def get_keypoints_on_image(img, corrected_normalized_keypoints,camera_params):
    """
    detects keypoints on image, with image warp and all the stuff, 
    returns points' coords on original image 
    """
    res = utils.detect_apriltag(img,camera_params)
    
    img_warped, desired_corners,tform = warp_img(img, res)
    res_warp = utils.detect_apriltag(img_warped,camera_params)
    warped_keypoints = utils.get_keypoints_on_warped_image(res_warp, corrected_normalized_keypoints)
    desired_corners = utils.get_straight_corners(res_warp)
#     print("desired_corners", desired_corners)
    
    # tform = ProjectiveTransform()
    # estimate(src, dst)
    # tform.estimate(c_to, c_from)

    str_project = get_projection(desired_corners, res[0].corners)
    print(str_project.params)
    points_on_image = [str_project.params @ np.array([p[0], p[1], 1]) for p in warped_keypoints]
    for p in points_on_image:
        p/= p[2]
    
    return np.array(points_on_image)
 
def warp_img(image, res):
    """warps image so marker would be square"""
    desired_corners = utils.get_straight_corners(res)
    print(desired_corners, res[0].corners)
    tform = ProjectiveTransform()
    inverse_tf = tform.estimate(desired_corners, res[0].corners)

    image_warped = warp(image, inverse_tf)
    return image_warped, desired_corners, tform


def draw_corners(img, r, show=False):
    """draws marker corners, returns them"""
    if isinstance(r, np.ndarray):
        imagePoints = [r]
    else:
        imagePoints = r.corners.reshape(1,4,2)
    
    for corner in range(np.size(imagePoints[0],axis=0)):
        center = ((int(imagePoints[0][corner][0]),int(imagePoints[0][corner][1])))
        cv2.circle(img, center, 3, (0,0,255), -1)
    
    if show:
        cv2.imshow('img', color_image)
    return imagePoints[0]

In [30]:
# img = cv2.imread(img_with_marker_name)
img_with_marker_name = 'marker_close_new_2.jpg'

# img = cv2.imread(img_with_marker_name)

# utils.show_image_with_points(img,anchor_ground_truth, "kpts_on_marker")
img = cv2.imread(img_with_marker_name)
res = utils.detect_apriltag(img,camera_params)
print(res)
img_warped, desired_corners,tform = warp_img(img,res)

utils.show_image_with_points(img_warped,anchor_ground_truth, "img_warped")

cv2.destroyAllWindows()

[Detection object:
tag_family = b'tagStandard52h13'
tag_id = 48701
hamming = 0
decision_margin = 58.5472297668457
homography = [[   0.37483707 -107.18212331  602.57440465]
 [ 105.41946277    1.02560177  330.03666681]
 [  -0.00147215   -0.00122736    1.        ]]
center = [602.57440465 330.03666681]
corners = [[494.89630127 225.58758545]
 [497.10906982 437.66320801]
 [710.30523682 434.53689575]
 [707.47186279 222.98963928]]
pose_R = [[ 0.01351805 -0.99984746 -0.01105942]
 [ 0.99968805  0.01328193  0.02115189]
 [-0.02100177 -0.01134191  0.9997151 ]]
pose_t = [[-0.0059953 ]
 [-0.00740351]
 [ 0.14366104]]
pose_err = 1.8576139409624874e-07
]
[[496 224]
 [496 436]
 [708 224]
 [708 436]] [[494.89630127 225.58758545]
 [497.10906982 437.66320801]
 [707.47186279 222.98963928]
 [710.30523682 434.53689575]]


TypeError: 'bool' object is not callable

In [31]:
cv2.destroyAllWindows()

In [ ]:
# def warp_img(image, res):
#     """warps image so marker would be square"""
#     desired_corners = get_straight_corners(res)

#     tform = get_projection(desired_corners, res[0].corners)
#     image_warped = warp(image, tform)
#     return image_warped, desired_corners, tform

utils.show_image_with_points(img,points_on_image, "kpts_on_marker")

# Calibration

In [9]:
cv2.destroyAllWindows()

# we take photos from as close as possible
img_with_marker_name = 'marker_close_new_2.jpg'


img = cv2.imread(img_with_marker_name)

res = utils.detect_apriltag(img, camera_params)
img_warped, desired_corners,tform = utils.warp_img(img, res)

# print(str_project.params)
points_on_image = [tform.params @ np.array([p[0], p[1], 1]) for p in anchor_ground_truth]
for p in points_on_image:
    p/= p[2]
points_on_image = np.array(points_on_image)
res_warp = utils.detect_apriltag(img_warped,camera_params)

anchor_on_image = utils.get_keypoints_on_warped_image(res_warp, normalized_anchors)
print("anchor difference in pix", anchor_on_image-anchor_ground_truth)

# calculates corrected points from 1 shot
kpts_transformed,corrected_normalized_keypoints,transformed_anchors_points = utils.get_corrected_normalized_keypoints(res_warp,
                                        normalized_anchors,
                                        anchor_ground_truth,
                                        normalized_keypoints)

cv2.destroyAllWindows()
# corrected_normalised_keypoints is what we need

anchor difference in pix [[-4.62428663  7.01455117]
 [-4.26832825  3.02692779]
 [-4.85491859  3.94094286]
 [-4.93881626  6.07484948]
 [-4.18292562  8.95076093]
 [-5.48483767  7.73925008]
 [-4.16732841  8.71351528]]


error: OpenCV(4.5.4) ../modules/calib3d/src/ptsetreg.cpp:1022: error: (-215:Assertion failed) count >= 0 && to.checkVector(2) == count in function 'estimateAffine2D'


In [7]:
points_on_image[:,:2] - anchor_ground_truth

array([[ 0.50720594, -0.01005746],
       [-0.1740037 ,  0.52221415],
       [ 0.06308917, -0.39180091],
       [ 0.17123305,  0.92964424],
       [ 0.71634745, -0.94626722],
       [ 1.04250572,  0.46624145],
       [ 1.37549899, -0.50802375]])

In [7]:
# kpts_transformed - normalized_keypoints

keypoints = utils.get_keypoints_on_image(img_m, corrected_normalized_keypoints, camera_params)
print(keypoints[:,:2] - kpts_transformed)

[[ 1.01357425  0.00354467 -6.30649152]
 [-0.00969867  0.99690554  8.6742203 ]
 [ 0.00001161 -0.00001392  1.        ]]
[[-0.41704922  1.79105774]
 [-0.31745568  1.16642855]
 [-0.35849922  2.73184072]
 [ 0.20766572 -0.13745943]
 [-0.00242317  1.96303519]
 [ 0.28353443  0.56524883]
 [ 0.03761471  2.66235291]
 [ 0.74909092 -0.36998423]
 [ 0.3054197   3.21134274]
 [ 1.56076472 -1.19939394]
 [ 2.32982278  2.33707719]
 [ 3.32199326 -0.23767849]
 [ 2.40864421  2.40025547]
 [ 3.44826556 -0.25780689]]


In [9]:
print(keypoints[:,:2] - kpts_transformed)

[[-0.41704922  1.79105774]
 [-0.31745568  1.16642855]
 [-0.35849922  2.73184072]
 [ 0.20766572 -0.13745943]
 [-0.00242317  1.96303519]
 [ 0.28353443  0.56524883]
 [ 0.03761471  2.66235291]
 [ 0.74909092 -0.36998423]
 [ 0.3054197   3.21134274]
 [ 1.56076472 -1.19939394]
 [ 2.32982278  2.33707719]
 [ 3.32199326 -0.23767849]
 [ 2.40864421  2.40025547]
 [ 3.44826556 -0.25780689]]


In [5]:
# marker_name = f'/home/viacheslav/jupyter_notebooks/data/far_photos_with_marker/{img_num}.jpg'
# no_marker_name = f'/home/viacheslav/jupyter_notebooks/data/far_photos/{img_num}.jpg'


In [ ]:
# img_w, warp_res, _ = utils.warp_img(img_m, utils.detect_apriltag(img_m, camera_params))

In [ ]:
# cv2.imshow('w', img_w)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[[ 1.01357425  0.00354467 -6.30649152]
 [-0.00969867  0.99690554  8.6742203 ]
 [ 0.00001161 -0.00001392  1.        ]]


array([[570.28690273, 174.51486652,   1.        ],
       [624.65008891, 173.78645991,   1.        ],
       [477.36310545, 214.69109366,   1.        ],
       [718.31686353, 211.35433896,   1.        ],
       [539.80449357, 221.77588628,   1.        ],
       [656.34930348, 220.15126262,   1.        ],
       [475.66496524, 256.16723767,   1.        ],
       [721.13235294, 252.65056996,   1.        ],
       [429.47587304, 317.45661308,   1.        ],
       [768.656754  , 312.36021777,   1.        ],
       [512.88752656, 461.33019207,   1.        ],
       [689.6869646 , 458.37748774,   1.        ],
       [510.77815285, 470.75196659,   1.        ],
       [692.04307076, 467.70504463,   1.        ]])

In [ ]:
# import dt_apriltags as apriltag
# apriltag.Detector(families='tagStandard52h13').detect(cv2.cvtColor(cv2.imread('marker.jpg'), cv2.COLOR_BGR2GRAY), estimate_tag_pose=True, camera_params=camera_params, tag_size=.04)

In [ ]:
img_nm = cv2.imread(no_marker_name)

for kp in transformed_anchors_points:
    cv2.circle(img_nm, (int(kp[0]), int(kp[1])), 2, (0,0,255), -1)
    
# for kp in anchor_ground_truth:
#     cv2.circle(img_nm, (int(kp[0]), int(kp[1])), 2, (255,0,0), -1)
cv2.imshow('img with keypoints', img_nm)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
cv2.destroyAllWindows()

In [ ]:
cv2.Canny??

In [ ]:
img = cv2.imread('data/marker_close_new.jpg')
res = utils.detect_apriltag(img, camera_params)

In [ ]:
for c in res[0].corners:
    cv2.circle(img, (int(c[0]), int(c[1])), 2, (0,0, 255))
    
cv2.imshow('corners',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img_without_marker_name = 'no_marker_close_new_2.jpg'
img = cv2.imread(img_without_marker_name)

# edges = cv2.Canny(img,40,100, True)
# cv2.imshow('edges', edges)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# kernel = np.ones((5,5),np.float32)/25
# dst = cv2.filter2D(edges,-1,kernel)

In [ ]:
def circles_by_order(circles):
    horizontality_thres = 7
    new_order = np.zeros_like(circles)
    mean = np.mean(circles, axis=0)
    
    def dist(a, b):
        return abs(a[0]-b[0]) + abs(a[1]-b[1])
    
    dists = [dist(circle, mean) for circle in circles]
    center = np.argmin(dists)
    new_order[0] = circles[center]
    
    for i, circle in enumerate(circles):
        if i == center: continue
        if circle[0] < new_order[0][0]:
            if abs(circle[1] - new_order[0][1]) < horizontality_thres:
                new_order[3] = circle
            elif circle[1] < new_order[0][1]:
                new_order[1] = circle
            else:
                new_order[5] = circle
        else:
            if abs(circle[1] - new_order[0][1]) < horizontality_thres:
                new_order[4] = circle
            elif circle[1] < new_order[0][1]:
                new_order[2] = circle
            else:
                new_order[6] = circle
    return new_order

circles = cv2.HoughCircles(dst, cv2.HOUGH_GRADIENT, dp=1.5, maxRadius=45, minRadius=15, 
                           minDist=30, param2=80)
out = img.copy()
print(circles)
for x, y, r in circles[0]:
    cv2.circle(out, (int(x), int(y)), int(r), (0, 255, 0), 4)
    cv2.rectangle(out, (int(x) - 2, int(y) - 2), (int(x) + 2, int(y) + 2), (0, 128, 255), -1)
cv2.imshow('circles', out)
cv2.waitKey(0)
cv2.destroyAllWindows()
circles = np.array([(int(c[0]), int(c[1])) for c in circles[0]])
anchor_ground_truth = circles_by_order(circles)
anchor_ground_truth